In [ ]:
import mbuild as mb
import os
from parmed import load_file 
import re
from mbuild.lib.atoms import H
import numpy as np
import random
import pubchempy as pcp
from rdkit import Chem 
from  Main_functions import *

Using part of the Mbuild functionality and the Polymer class, let's build and visualize a PVA polymer chain with 12 repeat units.
For this, first we will build the repeat unit of PVA:
1. First we will create an ethanol molecule
2. We will remove atoms and add ports in order to create the monomer that will constitute the polymer chain
3. The ports will be shown in the updated repeat unit

In [65]:
main_name = "ethanol"
MB_molecule_to_create_monomer = Alcohol(chain_length=2)                                 
first_port_monomer = "COH"
second_port_monomer = "CH3"

main_polymer_bead, end_bead = Builder_pipeline_Polymer(main_name,
                                                       first_port = first_port_monomer,
                                                       second_port = second_port_monomer,
                                                       add_ports = True,
                                                       rigid_port = False,
                                                       fetch_compound = False,
                                                       MB_compound = MB_molecule_to_create_monomer,
                                                       factor_length = 1)


repeat_units = 12 # Repeat Units in polymer chain
polymer_compound_ = mb.recipes.Polymer(monomers=[main_polymer_bead])
polymer_compound_.build( n=repeat_units,)
main_polymer_bead.visualize(show_ports = True)

Polymer loaded as mb compound, no query needed
COH
The port is type: COH
Added ports to the molecule
Added 2 ports to the molecule


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Alternatively, the polymer repeat unit can be prepared as a .mol2 or .pdb file and loaded into an mBuild compound, which can then be passed to the Polymer class.

In this case we will build a repeat unit of PVA that is saturated with hydrogens, as one of the hydrogens at each end of the polymer must be removed to create
the bond between repeat units

In [66]:
repeat_unit_from_file = mb.load('files/PVA_repeat_unit.mol2')
repeat_unit_from_file.visualize()

/raid6/homes/ordoriaf/miniconda3/envs/andres/lib/python3.10/site-packages/mdtraj/core/trajectory.py:441: UserWarning: top= kwargs ignored since this file parser does not support it
  warnings.warn("top= kwargs ignored since this file parser does not support it")


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

When loading a compound from a file, ports must be added to define the connection points. These ports allow the Polymer class to determine where bonds should be formed between repeat units.

For more information refer to: https://mbuild.mosdef.org/en/stable/getting_started/quick_start/polymer_example.html

In this case, the bond takes place between:

    1. Carbon atom with three bonds (-H, -H, -C); CH3
    2. Carbon atom with three bonds (-H, -O, -C); COH2

For this we will define the ports manually in a dictionary:

In [67]:
ports_to_add = {
    "CH3" : [
        ["C",  # The first atom is the "anchor" or base atom 
          "H", "H", "H", "C"], # Bonded atoms to Carbon 
        ["H"], # Atom to remove and create port
        [4], # Total number of bonds in carbon atom 
        ],

     "COH2" : [
        ["C", # The first atom is the "anchor" or base atom 
          "O", "H", "H", "C"], # Bonded atoms to Carbon 
        ["H"], # Atom to remove and create port
        [4], # Total number of bonds in carbon atom 
        ],
}

Once we define the ports that are added, we will add labels so the Polymer class can determine where the bonds should be created

In [68]:
compound_1_port = Add_port_CR(repeat_unit_from_file,
                                type = "CH3",
                                name = "up",
                                manual_ports= True,
                                ports_dict = ports_to_add)    
    
print("Added ports to the molecule")
tmp_cmp = mb.clone(compound_1_port)

#Adding second port to the repeat unit        
compound_with_2_ports = Add_port_CR(tmp_cmp,
                                    type = "COH2",
                                    name = "down",
                                    manual_ports= True,
                                    ports_dict = ports_to_add)      
print("Added 2 ports to the molecule")

Added ports to the molecule
Added 2 ports to the molecule


In [69]:
compound_with_2_ports.visualize(show_ports= True)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

As shown in the previous visualization, ports are added at the positions where hydrogens were originally located. However, if the placement of ports requires more caution or the molecular chemistry is more complex, the initial repeat unit can be modified to include “dummy” atoms, making the addition of ports easier.

In [70]:
repeat_unit_from_file_dummy_atoms = mb.load('files/PVA_repeat_unit_dummy_atoms.mol2')
repeat_unit_from_file_dummy_atoms.visualize()

/raid6/homes/ordoriaf/miniconda3/envs/andres/lib/python3.10/site-packages/mdtraj/core/trajectory.py:441: UserWarning: top= kwargs ignored since this file parser does not support it
  warnings.warn("top= kwargs ignored since this file parser does not support it")


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In this repeat unit file, a selected set of hydrogens was replaced with boron atoms, allowing the placement of ports to be directly controlled.

In [71]:
ports_to_add_Dummy_atoms = {
    "CH2-B" : [
        ["C",  # The first atom is the "anchor" or base atom 
          "B", "H", "H", "C"], # Bonded atoms to Carbon 
        ["B"], # Atom to remove and create port
        [4], # Total number of bonds in carbon atom 
        ],

     "COH-B" : [
        ["C", # The first atom is the "anchor" or base atom 
          "O", "B", "H", "C"], # Bonded atoms to Carbon 
        ["B"], # Atom to remove and create port
        [4], # Total number of bonds in carbon atom 
        ],
}

In [72]:
compound_1_port_DA = Add_port_CR(repeat_unit_from_file_dummy_atoms,
                                type = "CH2-B",
                                name = "up",
                                manual_ports= True,
                                ports_dict = ports_to_add_Dummy_atoms)    
    
print("Added ports to the molecule")
tmp_cmp_DA = mb.clone(compound_1_port_DA)

#Adding second port to the repeat unit        
compound_with_2_ports_DA = Add_port_CR(tmp_cmp_DA,
                                    type = "COH-B",
                                    name = "down",
                                    manual_ports= True,
                                    ports_dict = ports_to_add_Dummy_atoms)      
print("Added 2 ports to the molecule")

Added ports to the molecule
Added 2 ports to the molecule


In [73]:
compound_with_2_ports_DA.visualize(show_ports= True)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Finally we take the repeat unit, and pass it to the Polymer class defining the number of repeat units:

In [74]:
repeat_units = 12 # Repeat Units in polymer chain
polymer_compound_ = mb.recipes.Polymer(monomers=[compound_with_2_ports_DA])
polymer_compound_.build( n=repeat_units,)
polymer_compound_.visualize()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

The compound can now be saved to a mol2 file or pdb:

In [75]:
for suffix in ["mol2", "pdb"]:
    polymer_compound_.save(f"output/12_RU_PVA.{suffix}")
